# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Ishaan Bhondele, 23-745-862 and emal address>

In [2]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [12]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector and covariance matrix for the multivariate normal distribution
mean = np.random.rand(N)
A = np.random.rand(N, N)
cov = np.dot(A, A.T) #ensure PSD

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])

# Compute the vector of expected returns (mean returns) from df
expected_return = ExpectedReturn()
expected_return.estimate(X=df)
q = expected_return.vector
# Compute the covariance matrix from df
covariance = Covariance()
covariance.estimate(X=df)

P = covariance.matrix

# Display the results
print("Vector of expected returns (q):")
print(q)

print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
Asset_1      2.777558
Asset_2      2.329994
Asset_3      2.915305
Asset_4      2.829719
Asset_5      2.589311
               ...   
Asset_96     2.341384
Asset_97     2.477544
Asset_98     2.850532
Asset_99     2.750184
Asset_100    2.431605
Length: 100, dtype: float64

Covariance matrix (P):
             Asset_1    Asset_2    Asset_3    Asset_4    Asset_5    Asset_6  \
Asset_1    36.518327  24.020607  26.011403  24.334433  24.303544  21.991360   
Asset_2    24.020607  33.144129  25.764270  23.638660  26.107059  22.370053   
Asset_3    26.011403  25.764270  36.154073  25.715061  26.186684  22.253324   
Asset_4    24.334433  23.638660  25.715061  31.753181  25.641839  20.716205   
Asset_5    24.303544  26.107059  26.186684  25.641839  35.110772  22.436245   
...              ...        ...        ...        ...        ...        ...   
Asset_96   25.362066  23.546731  24.528489  23.263203  22.647548  18.873752   
Asset_97   24.685864  26.627312  26.721754

c:\Users\IAB\OneDrive\CODE\QPMP\Assignment_1\qpmwp-course\.venv\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [ ]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget(rhs=1.0)

# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower=0.0, upper=0.2)

# Add linear constraints
group1_ids = df.columns[:30]
g1 = pd.Series(np.ones(len(group1_ids)), index=group1_ids)
constraints.add_linear(g_values=g1, sense="<=", rhs=0.3, name="group1")

# For the second group (assets 31 to 60)
group2_ids = df.columns[30:60]
g2 = pd.Series(np.ones(len(group2_ids)), index=group2_ids)
constraints.add_linear(g_values=g2, sense="<=", rhs=0.4, name="group2")

# For the third group (assets 61 to 100)
group3_ids = df.columns[60:]
g3 = pd.Series(np.ones(len(group3_ids)), index=group3_ids)
constraints.add_linear(g_values=g3, sense="<=", rhs=0.5, name="group3")

AttributeError: 'Constraints' object has no attribute 'head'

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [ ]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
#<your code here>

Print and visualize the results

In [5]:
#<your code here>

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        #<your code here>

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            #<your code here>
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
#<your code here>

# Instantiate the MinVariance class
#<your code here>

# Prepare the optimization data and prepare the optimization problem
#<your code here>

# Solve the optimization problem and print the weights
#<your code here>